# Telegram Bot

Для бота нужно установить соответствующую библиотеку

https://pytba.readthedocs.io/en/latest/index.html

`! pip install pyTelegramBotAPI`

In [2]:
import json
import time
import requests


import telebot

In [95]:
import inspect


CONFIG_PATH = '.bot_data.json'

with open(CONFIG_PATH, 'r') as f:
    BOT_DATA = json.load(f)
    
bot = telebot.TeleBot(BOT_DATA['token'])

In [96]:
print(inspect.getsource(bot.message_handler))  # можно посмотреть на код функций

    def message_handler(self, commands: Optional[List[str]]=None, regexp: Optional[str]=None, func: Optional[Callable]=None,
                    content_types: Optional[List[str]]=None, chat_types: Optional[List[str]]=None, **kwargs):
        """
        Handles New incoming message of any kind - text, photo, sticker, etc.
        As a parameter to the decorator function, it passes :class:`telebot.types.Message` object.
        All message handlers are tested in the order they were added.

        Example:

        .. code-block:: python3
            :caption: Usage of message_handler

            bot = TeleBot('TOKEN')

            # Handles all messages which text matches regexp.
            @bot.message_handler(regexp='someregexp')
            def command_help(message):
                bot.send_message(message.chat.id, 'Did someone call for help?')

            # Handles messages in private chat
            @bot.message_handler(chat_types=['private']) # You can add more chat types
 

In [102]:
%%writefile csda_bot.py

import json
import time
import requests

import telebot

DAY2MONTH_RU = {
    1: 'Январь',
    2: 'Февраль',
    3: 'Март',
    4: 'Апрель',
    5: 'Май',
    6: 'Июнь',
    7: 'Июль',
    8: 'Август',
    9: 'Сентябрь',
    10: 'Октябрь',
    11: 'Ноябрь',
    12: 'Декабрь',
}

DAY2MONTH_EN = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December",
}

CONFIG_PATH = '.bot_data.json'

with open(CONFIG_PATH, 'r') as f:
    BOT_DATA = json.load(f)
    
API_TOKEN = BOT_DATA['token']


def add_date_remainder(day_of_month: int):
    '''
    Add day_of_month remainder for English language
    (contributed by strudent)
    '''
    return f'the {day_of_month}{"st" if (remainder := day_of_month % 10) == 1 and day_of_month != 11 else "nd" if remainder == 2 and day_of_month != 12 else "rd" if remainder == 3 and day_of_month != 13 else "th"}'

def generate_data_phrase(location: str='Armenia', lang: str='en'):
    time_struct = time.gmtime()
        
    if lang == 'ru':    
        date_str = f'{location}, {time_struct.tm_mday} число, {DAY2MONTH_RU[time_struct.tm_mon]}, {time_struct.tm_hour:02}:{time_struct.tm_min:02}:{time_struct.tm_sec:02}.'
    else:
        date_str = f'Today in {location}, {add_date_remainder(time_struct.tm_mday)} of {DAY2MONTH_EN[time_struct.tm_mon]}, {time_struct.tm_hour:02}:{time_struct.tm_min:02}:{time_struct.tm_sec:02}.'
                
    return date_str


def ask_balaboba(location: str = 'Russia',
                 lang: str = 'en',
                 url: str = 'https://yandex.ru/lab/api/yalm/text3',
                 delay: int = 0):
    time.sleep(delay)

    query = generate_data_phrase(location=location, lang=lang)

    headers = {
        'Content-Type': 'application/json',
        'Origin': 'https://yandex.com',
        'Referer': 'https://yandex.com',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:105.0) Gecko/20100101 Firefox/105.0',
    }

    payload = json.dumps({
        "query": query,
        "intro": 28,  # 28
        "filter": 1,
    })

    response = requests.request('POST', url, headers=headers, data=payload)

    if response.ok:
        text = response.json()['text']
        text = 'There is nothing.' if not text else text

        return f'{query} {text}'

    else:
        return f'No story. Status code {response.status_code}'


bot = telebot.TeleBot(API_TOKEN)

@bot.message_handler(commands=['start'])
def start(message):
    markup = telebot.types.ReplyKeyboardMarkup(resize_keyboard=True)
    markup.add(telebot.types.KeyboardButton("Story Armenia En"))
    markup.add(telebot.types.KeyboardButton("Story Армения Ru"))
    markup.add(telebot.types.KeyboardButton("Story Russia En"))
    markup.add(telebot.types.KeyboardButton("Story Россия Ru"))
    
    bot.send_message(
        message.chat.id,
        'Press to get an amazing story!',
        reply_markup=markup
    )

    
@bot.message_handler(content_types=['text'])
def answer_message(message):
    try:
        command = message.text.strip().split()[0]
        if command.lower() == 'story':
            country, lang = message.text.strip().split()[1:]

            if country and lang:
                story = ask_balaboba(location=country, lang=lang.lower(), delay=2)
            else:
                story = ask_balaboba(delay=2)
            
            reply = f'Here is your story:\n```\n{story}\n```'
            bot.send_message(BOT_DATA['channel'], reply, parse_mode='Markdown')
        
        else:
            story = f'I see your "{message.text}", so what?'
    
    except ValueError:
            story = f'I see your "{message.text}", are you sure you wanted to formulate it in this way?'
            
    reply = f'Here is your story:\n```\n{story}\n```'
    bot.reply_to(message, reply, parse_mode='Markdown')
    
    
bot.infinity_polling()

Overwriting csda_bot.py
